# Stock predication

Анализ ценных бумаг преследует цель определить инвестиционное качество той или иной ценной бумаги и ее цены.

В нашем случае, это еще и предсказание цены на основе прошлых данных.


Анализ ценных бумаг преследует две цели:
Во-первых, он позволяет так представить важные факты, касающиеся акций или других ценных бумаг, чтобы это было удобно текущему или потенциальному владельцу этих активов.
Во-вторых, он позволяет сделать основанное на фактах и прикладных стандартах заключение о безопасности и привлекательности данных активов при текущей или предполагаемой цене.

# Import libraries and packages

In [135]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
#from newsapi import NewsApiClient

# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle

# from google.colab import drive

# ! pip install plotlyы
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

historical_prices = pd.read_csv("./content/UPRO.csv")

In [136]:
historical_prices.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,09/22/2023,2.293,2.300,2.322,2.260,161.50M,-0.26%
1,09/21/2023,2.299,2.250,2.335,2.231,330.49M,1.86%
2,09/20/2023,2.257,2.290,2.333,2.126,478.39M,-1.27%
3,09/19/2023,2.286,2.407,2.417,2.260,279.69M,-4.63%
4,09/18/2023,2.397,2.462,2.475,2.385,164.11M,-1.68%


# Rename columns

In [137]:
historical_prices.rename(columns={"Date":"date","Open":"open","High":"high","Low":"low","Close":"close", "Adj Close":"adj close", "Price":"close", "Volume":"volume", "Vol.":"volume", "volume":"volume"}, inplace= True)


# def multiply_vol(value):
#     if "M" in value:
#         return float(value.replace("M", "")) * 1000000
#     elif "B" in value:
#         return float(value.replace("B", "")) * 1000000000
#     else:
#         return float(value)

# def replace(value):
#     if "." in value:
#         return str(value.replace(".", "."))
#     else:
#         return str(value)


# historical_prices["volume"] = historical_prices["volume"].apply(replace)
# historical_prices["low"] = historical_prices["low"].apply(replace)


# historical_prices["volume"] = historical_prices["volume"].apply(multiply_vol)

historical_prices.head()

,date,close,open,high,low,volume,Change %
0,09/22/2023,2.293,2.300,2.322,2.260,161.50M,-0.26%
1,09/21/2023,2.299,2.250,2.335,2.231,330.49M,1.86%
2,09/20/2023,2.257,2.290,2.333,2.126,478.39M,-1.27%
3,09/19/2023,2.286,2.407,2.417,2.260,279.69M,-4.63%
4,09/18/2023,2.397,2.462,2.475,2.385,164.11M,-1.68%


#**Preprocessing data**

## Cheking null and na value

In [138]:
# Checking null value
historical_prices.isnull().sum()

date        0
close       0
open        0
high        0
low         0
volume      0
Change %    0
dtype: int64

In [139]:
# Checking na value
historical_prices.isna().any()

date        False
close       False
open        False
high        False
low         False
volume      False
Change %    False
dtype: bool

In [140]:
# Drop na value
historical_prices.dropna(inplace=True)
historical_prices.isna().any()

date        False
close       False
open        False
high        False
low         False
volume      False
Change %    False
dtype: bool

In [141]:
# Checking null value
historical_prices.isnull().sum()

date        0
close       0
open        0
high        0
low         0
volume      0
Change %    0
dtype: int64

## Check datatype of each column

In [142]:
print("Date column data type: ", type(historical_prices['date'][0]))
print("Open column data type: ", type(historical_prices['open'][0]))
print("Close column data type: ", type(historical_prices['close'][0]))
print("High column data type: ", type(historical_prices['high'][0]))
print("Low column data type: ", type(historical_prices['low'][0]))

Date column data type:  <class 'str'>
Open column data type:  <class 'numpy.float64'>
Close column data type:  <class 'numpy.float64'>
High column data type:  <class 'numpy.float64'>
Low column data type:  <class 'numpy.float64'>


## Convert date, close, open from string

In [143]:
historical_prices['date'] = pd.to_datetime(historical_prices.date, format="%m/%d/%Y")
# historical_prices['open'] = pd.to_numeric(historical_prices.open.str.replace(',', '.'))
# historical_prices['close'] = pd.to_numeric(historical_prices.close.str.replace(',', '.'))
# historical_prices['high'] = pd.to_numeric(historical_prices.high.str.replace(',', '.'))
# historical_prices['low'] = pd.to_numeric(historical_prices.low.str.replace(',', '.'))
# historical_prices['volume'] = historical_prices.volume.str.replace(',', '.')
historical_prices.head()

,date,close,open,high,low,volume,Change %
0,2023-09-22,2.293,2.300,2.322,2.260,161.50M,-0.26%
1,2023-09-21,2.299,2.250,2.335,2.231,330.49M,1.86%
2,2023-09-20,2.257,2.290,2.333,2.126,478.39M,-1.27%
3,2023-09-19,2.286,2.407,2.417,2.260,279.69M,-4.63%
4,2023-09-18,2.397,2.462,2.475,2.385,164.11M,-1.68%


## Sorting dataset by date format

In [144]:
historical_prices.sort_values(by='date', inplace=True)
historical_prices.head()
historical_prices.shape

(4077, 7)

# **EDA - Exploratory Data Analysis (Исследовательский анализ данных)**

Попробуем ответить на следующие вопросы:
1.  Какая динамика изменения максимальной и минимальной цены  между месяцами за последний год?
2.  Какой день был самый волантильный за последний год?
3.  Какой тренд (рост или падание) за последние время  акция?
4.  Когда был последний день смены тренда?
5.  Какой день был самый волантильный за последний год?

В каждом исследовании представлена небольшая справка.

Get the duration of dataset (Получаем продолжительность набора данных)

In [145]:
print("Starting date: ",historical_prices.iloc[0][0])
print("Ending date: ", historical_prices.iloc[-1][0])
print("Duration: ", historical_prices.iloc[-1][0]-historical_prices.iloc[0][0])

Starting date:  2007-05-25 00:00:00
Ending date:  2023-09-22 00:00:00
Duration:  5964 days 00:00:00


## Monthwise comparision between Stock actual, open and close price (Ежемесячное сравнение между ценой открытия и закрытия)

In [146]:
monthvise= historical_prices.groupby(historical_prices['date'].dt.strftime('%B'))[['open','close']].mean().sort_values(by='close')
monthvise.head()

,open,close
date,,
May,2.361321,2.358840
August,2.376997,2.378560
July,2.386675,2.388875
December,2.396783,2.397128
March,2.393648,2.398492


In [147]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock actual open and close price')
fig.show()

 Результат:  
 Наблюдаем, что май был самым благоприятным месяцем для фиксации прибыли. Январь 2021 был самым удачным для покупки акции.

## Monthwise High and Low stock price (Месячная максимальная и минимальная цена акций)

In [148]:
historical_prices.groupby(historical_prices['date'].dt.strftime('%B'))['low'].min()

date
April        0.468
August       0.936
December     0.233
February     0.331
January      0.277
July         0.846
June         0.853
March        0.404
May          0.840
November     0.187
October      0.398
September    0.871
Name: low, dtype: float64

In [149]:
monthvise_high= historical_prices.groupby(historical_prices['date'].dt.strftime('%B'))['high'].max()
monthvise_low= historical_prices.groupby(historical_prices['date'].dt.strftime('%B'))['low'].min()

In [150]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

   Итог:  Предположаительно июнь и декабарь были самыми волантильными месяцами, когда была большая разница между максимальной и минимальной ценой акции.  

## Momentum Strength
        Momentum is the velocity of price changes in a stock. It is used by investors to define if a stock can exhibit bullish trend, rising price, or bearish trend where the price is steadily falling.
        Моментум – это скорость изменения цены акции. Он используется инвесторами, чтобы определить, может ли акция демонстрировать бычий тренд, рост цены или медвежий тренд, когда цена неуклонно падает.

Если линия выше 0 - то тренд роста.
Если линия ниже 0 - то тренд падения.
Большой угол измения линии показывает силу тренда и локальную точку смены тренда.

In [151]:
df_cl = np.asarray(historical_prices['close'])
momentum = np.subtract(df_cl[10:], df_cl[:-10])
names = cycle(['Stock Close Price', 'momentum'])

fig = px.line(historical_prices, x=historical_prices.date[10:], y=[historical_prices['close'][10:],momentum],
             labels={'date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False,zeroline=True, zerolinewidth=2, zerolinecolor='Black')

fig.show()

Итог: В июне после макимального роста начался тренд падения акции, это связано резким увелечением ключевой ставки Федеральной системы США.
Аналогичная ситуация 23-26 сентября.
График изменения ключевой ставки доступен по ссылке: https://ru.investing.com/economic-calendar/interest-rate-decision-168

## Arron indicator
        The Arron indicator is composed of two lines. 
        Considering a time range, an up line measures the number of periods since the highest price in the range, and a down line which measures the number of periods since the lowest price.
        Aroon indicates a bullish behavior when the Aroon up is above the Aroon down. 
        The opposite case indicates a bearish price behavior, and when the two lines cross each other can signal a trend changes.
        
        Что такое индикатор Aroon? Индикатор Aroon — это технический индикатор, который используется для определения трендовых изменений цены актива, а также силы этого тренда. 
        По сути, индикатор измеряет время между максимумами и время между минимумами за определенный период времени.

In [152]:
        period=7
        df_cl1 = np.asarray(historical_prices['close'])
        aroon_down=[(100/period)*
                 (period-np.argmax(df_cl1[t-period:t])) 
                 for t in range(period, len(df_cl1))]
        aroon_up=[(100/period)*
                   (period-np.argmin(df_cl1[t-period:t])) 
                   for t in range(period, len(df_cl1))]

names_1 = cycle(['Stock Close Price'])
names_2 = cycle(['aroon_up','aroon_down'])
fig = px.line(historical_prices, x=historical_prices.date[period:], y=[historical_prices['close'][period:]],
             labels={'date': 'Date','value':'Stock value'})
figs = px.line(historical_prices, x=historical_prices.date[period:], y=[aroon_up,aroon_down],
             labels={'date': 'Date','value':'Stock value'})

figs.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
figs.for_each_trace(lambda t:  t.update(name = next(names_2)))
figs.update_xaxes(showgrid=False)
figs.update_yaxes(showgrid=False,zeroline=True, zerolinewidth=2, zerolinecolor='Black')

fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names_1)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False,zeroline=True, zerolinewidth=2, zerolinecolor='Black')

fig.show()
figs.show()

Итог: наблюдаем что смена тренда была 19-20 октября 2022, начался рост акции. И до сих пор продолжается тренд роста. Так же с помощью данного индикатора можно предсказать смену тренда.

## Volatility
        Volatility is a measurement of the variation of prices over time.
    
        Method returns:
            -   Volatility (annualized standard deviation) numerical value, in percent
            -   Daily log return of the Close price

Волатильность – это измерение изменения цен во времени.
    
        Метод возвращает:
            - Числовое значение волатильности (стандартное отклонение в годовом исчислении), в процентах
            - Ежедневный лог возврата цены закрытия

In [153]:
def volatility_result(volatility_percent):
    volatility_percent = float(volatility_percent)
    if 12 <= volatility_percent <= 20:
        result = "у акции нормальная волатильность"
    elif volatility_percent < 12:
        result = "у акции низкая волатильность"
    else:
        result = "у акции высокая волатильность"
    print(result)
       
historical_prices['Log_returns'] = np.log(historical_prices['close']/historical_prices['close'].shift())
volatility_year = historical_prices['Log_returns'].std()*252**.5
volatility_percent = str(round(volatility_year, 4) * 100)
print(volatility_percent)
        
volatility_result(volatility_percent)

historical_prices.head()

41.339999999999996
у акции высокая волатильность


,date,close,open,high,low,volume,Change %,Log_returns
4076,2007-05-25,2.650,2.790,2.790,2.501,1.16M,-6.03%,NaN
4075,2007-05-28,2.542,2.723,2.723,2.515,382.10K,-4.08%,-0.041608
4074,2007-05-29,2.350,2.500,2.559,2.251,542.60K,-7.55%,-0.078536
4073,2007-05-30,2.679,2.251,2.679,2.137,1.33M,14.00%,0.131028
4072,2007-05-31,2.401,2.679,2.700,2.401,2.47M,-10.38%,-0.109558


In [154]:
# Найдем индекс строки с максимальным значением 'Log_returns'
max_log_returns_index = historical_prices['Log_returns'].idxmax()
min_log_returns_index = historical_prices['Log_returns'].idxmin()

# Получим дату по этому индексу
date_with_max_log_returns = historical_prices.loc[max_log_returns_index, 'date']
date_with_min_log_returns = historical_prices.loc[min_log_returns_index, 'date']

print("Дата с самым большим значением 'Log_returns':", date_with_max_log_returns)

print("Дата с самым маленьким значением 'Log_returns':", date_with_max_log_returns)

# from newsapi import NewsApiClient
# # Init
# newsapi = NewsApiClient(api_key='9b9c326bccc64c099ea400316cf9ed5f')

# # /v2/top-headlines
# top_headlines = newsapi.get_top_headlines(q='russia',
#                                           sources='bbc-news,the-verge',
#                                           category='business',
#                                           language='en',
#                                           country='us')

# # /v2/everything
# all_articles = newsapi.get_everything(q='russia',
#                                       sources='bbc-news,the-verge',
#                                       domains='bbc.co.uk,techcrunch.com',
#                                       from_param='2009-05-06',
#                                       to='2009-05-06',
#                                       language='en',
#                                       sort_by='relevancy',
#                                       page=2)

# # /v2/top-headlines/sources
# sources = newsapi.get_sources()

Дата с самым большим значением 'Log_returns': 2009-05-06 00:00:00
Дата с самым маленьким значением 'Log_returns': 2009-05-06 00:00:00


In [155]:
fig = px.line(historical_prices, x=historical_prices.date, y=[historical_prices['Log_returns']],
             labels={'date': 'Date','value':'Volatility'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False,zeroline=True, zerolinewidth=2, zerolinecolor='Black')

fig.show()

Итог: 34% составляет волантильность данной акции за год, те разницам между макимальной и минимальнйо ценой акции составляет 34 процента.
Это является нормлаьным явлением. Акция очень стабильная. Волантильность 23-26 сентября связана с повышениам ставки ФРС, аналогичная ситация и в марте (первое повышение ключевоцй ставки)



## Trend comparision between stock price, open price, close price, high price, low price

In [156]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(historical_prices, x=historical_prices.date, y=[historical_prices['open'], historical_prices['close'], 
                                          historical_prices['high'], historical_prices['low']],
             labels={'date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

Итог: видно что в сентябре разница между ценами быстро изменялась, это и есть волантильность, которую расчитывали выше.



# Close price prediction preparation and preprocessing (Подготовка к прогнозу цены и предварительная обработка)



Make separate dataframe with close price. (Подготовка отдельного кадра данных с ценой закрытия)

In [157]:
closedf = historical_prices[['date','close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (4077, 2)


Plotting stock close price

In [158]:
fig = px.line(closedf, x=closedf.date, y=closedf.close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.6)
fig.update_layout(title_text='Stock close price chart', plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

## Normalizing / scaling close value between 0 to 1 (Нормализация/масштабирование близкого значения от 0 до 1)

In [159]:
close_stock = closedf.copy()
del closedf['date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(4077, 1)




## Split data for training and testing (Разделяем данные для обучения и тестирования)

Ratio for training and testing data is 65:35 (Соотношение данных для обучения и тестирования составляет 65:35)


In [160]:
training_size=int(len(closedf)*0.65)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (2650, 1)
test_data:  (1427, 1)


Create new dataset according to requirement of time-series prediction (Создание нового набора данных в соответствии с требованием прогнозирования временных рядов)

In [161]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [162]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (2634, 15)
y_train:  (2634,)
X_test:  (1411, 15)
y_test (1411,)


# Algorithms (Алгоритмы для предсказаний)

## Random Forest Regressor - RF

In [163]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [164]:
# Lets Do the prediction 

train_predict=regressor.predict(X_train)
test_predict=regressor.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (2634, 1)
Test data prediction: (1411, 1)


In [165]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

### Evaluation metrices RMSE, MSE and MAE (Среднеквадратическая ошибка (RMSE), среднеквадратическая ошибка (MSE) и средняя абсолютная ошибка (MAE) )

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

Среднеквадратическая ошибка (RMSE), среднеквадратическая ошибка (MSE) и средняя абсолютная ошибка (MAE) — это стандартный способ измерения ошибки модели при прогнозировании количественных данных.

In [166]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  0.020460341600224398
Train data MSE:  0.00041862557839787305
Test data MAE:  0.01412706529992405
-------------------------------------------------------------------------------------
Test data RMSE:  0.045653805942367835
Test data MSE:  0.0020842699970233807
Test data MAE:  0.030056116229624293




### Explained variance regression score (Объясненная оценка регрессии дисперсии)

The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

Показатель объясненной дисперсии объясняет дисперсию ошибок данного набора данных, и формула записывается следующим образом: Здесь и Var(y) — дисперсия ошибок прогнозирования и фактических значений соответственно. Крайне желательны оценки, близкие к 1,0, что указывает на лучшие квадраты стандартных отклонений ошибок.

In [167]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9989839242764326
Test data explained variance regression score: 0.9927960738493774


### R2 score for regression (Оценка R2 для регрессии)



R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

R-квадрат (R2) — это статистическая мера, представляющая долю дисперсии зависимой переменной, которая объясняется независимой переменной или переменными в регрессионной модели.

1 = Best
0 or < 0 = worse

In [168]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9989837289155457
Test data R2 score: 0.992791510492815


### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

Потеря регрессии Средняя потеря регрессии гамма-отклонения (MGD) и средняя потеря регрессии отклонения Пуассона (MPD)

In [169]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.00014165404884191801
Test data MGD:  0.0004980148909144363
----------------------------------------------------------------------
Train data MPD:  0.00020128371142701183
Test data MPD:  0.0009783039348844383


### Comparision between original stock close price vs predicted close price 
(Сравнение исходной цены закрытия акций с прогнозируемой ценой закрытия)

In [170]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (4077, 1)
Test predicted data:  (4077, 1)


Итог: цена прогназируемая повторяет оригинальные значения закрытия акции.

### Predicting next 15 days (Предсказание на следующие 15 дней)

In [171]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 15
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        
        yhat = regressor.predict(x_input)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = regressor.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  15


### Plotting last 15 days and next predicted 15 days

In [172]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30]


In [175]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

names = cycle(['Last 15 days close price','Predicted next 15 days close price'])

min_stock_price_preducation = min(next_predicted_days_value)

max_stock_price_preducation = max(next_predicted_days_value)

print("Min stock price preducation", min_stock_price_preducation)
print("Max stock price preducation", max_stock_price_preducation)

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Min stock price preducation 2.257
Max stock price preducation 2.569


### Plotting whole closing stock price with prediction (Построение всей цены акций на закрытие с прогнозированием)

In [ ]:
rfdf=closedf.tolist()
rfdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
rfdf=scaler.inverse_transform(rfdf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(rfdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.add_vrect(x0=closedf.size,x1=closedf.size)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

#   Итоги:
 Датасет выгружен с 5 декабря 2021г. по 5 декабря 2022г.
 Цена предсказана с 6 по 20 декабря 2022г.
 Цена закрытия акции  Shell с 5 декабря 2022г. по 15 декабря:
```
Date	        Open	High	Low	  Close  Adj Close**	Volume
Dec 15, 2022	56.76	56.90	55.78	56.15	56.15	6,085,901
Dec 14, 2022	57.38	57.48	56.54	56.76	56.76	4,325,588
Dec 13, 2022	57.37	57.74	57.15	57.38	57.38	5,091,499
Dec 12, 2022	56.06	56.56	55.88	56.36	56.36	4,491,861
Dec 09, 2022	56.27	56.78	55.61	55.61	55.61	4,280,227
Dec 08, 2022	57.21	57.44	56.37	56.59	56.59	4,571,387
Dec 07, 2022	56.96	57.32	56.35	56.35	56.35	5,444,943
Dec 06, 2022	57.52	58.03	56.51	56.68	56.68	6,428,901
Dec 05, 2022	58.77	58.97	57.10	57.40	57.40	5,917,006
```

Информация с сайта: 

https://finance.yahoo.com/quote/SHEL/ - источник датасета
https://www.kaggle.com/code/ysthehurricane/ - источник вдохновения

```
Волатильность:
34% составляет волантильность данной акции за год, те разницам между макимальной и минимальнйо ценой акции составляет 34 процента. Это является нормлаьным явлением. Акция очень стабильная. Волантильность 23-26 сентября связана с повышениам ставки ФРС, аналогичная ситация и в марте (первое повышение ключевой ставки)
```

```
Arron indicator:
Yаблюдаем что смена тренда была 20 октября 2022, начался рост акции. И до сих пор продолжается медленный тренд роста.
```

```
Momentum Strength:

В июне после макимального роста начался тренд падения акции, это связано резким увелечением ключевой ставки Федеральной системы США.
Аналогичная ситуация 23-26 сентября.

При следующем повышении (или новости о повышении) ключевой ставки ФРС США, акция будет терять свою стоимость.
```


